In [1]:
import pandas as pd
import datetime
import re
import yfinance as yf
from io import StringIO


In [2]:
table_txt = """
	Nombre	Variación diaria	Último Operado AR$	Cantidad	Tenencia US$
	AR$	0.00%	1,00	1.585,74	1,66
	US$ MEP	0.00%	955,88	3.744,58	3.744,58
	SPY	+7.19%	22.305,00	349,00	8.143,73
	DIA	+8.04%	17.721,00	86,00	1.594,34
	DISN	+7.81%	7.105,50	17,00	126,37
	INTC	+7.28%	9.504,50	10,00	99,43
	AL30	+0.42%	35.800,00	255,00	95,50
	GD30	-0.13%	38.000,00	200,00	79,51
"""

In [3]:
data = StringIO(table_txt)
df = pd.read_csv(data, delimiter='\t')
df.drop("Unnamed: 0", axis=1, inplace=True)
df

,Nombre,Variación diaria,Último Operado AR$,Cantidad,Tenencia US$
0,AR$,0.00%,"1,00","1.585,74","1,66"
1,US$ MEP,0.00%,"955,88","3.744,58","3.744,58"
2,SPY,+7.19%,"22.305,00","349,00","8.143,73"
3,DIA,+8.04%,"17.721,00","86,00","1.594,34"
4,DISN,+7.81%,"7.105,50","17,00","126,37"
5,INTC,+7.28%,"9.504,50","10,00","99,43"
6,AL30,+0.42%,"35.800,00","255,00","95,50"
7,GD30,-0.13%,"38.000,00","200,00","79,51"


In [4]:
def correct_str(input_str):
    input_str = input_str.replace(".", "")
    input_str = input_str.replace(",", ".")

    input_str = re.sub("[^0-9^.]", "", input_str)

    if input_str == '':
        input_str = 0.0
    return float(input_str)

In [5]:
for col in df.columns:
    if col != "Nombre":
        df[col] = df[col].apply(correct_str)

In [6]:
cedear_data = pd.read_excel("data/cedear_ratios_reloaded.xlsx", sheet_name=0)

In [7]:
df2 = df.merge(cedear_data, right_on="symbol", left_on="Nombre", how="left")

In [8]:
def find_price(stock_name):
    if stock_name is None:
        return 0.0
    else:
        return yf.download(stock_name, period="1d", interval="1d")["Adj Close"].iloc[0]

In [9]:
df2["base_symbol"] = df2["base_symbol"].astype(str)

In [10]:
df2["base_asset_price"] = df2["base_symbol"].apply(find_price)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [11]:
df2["this_asset_price"] = df2["base_asset_price"] / df2["ratio"]

In [12]:
df2.set_index("Nombre", inplace=True)

In [13]:
today_str = datetime.date.today().strftime("%m/%d/%Y")
ccl_dollar = pd.read_csv("data/dolar_ccl_historic.csv", index_col=0)

In [14]:
df2.loc["AR$","this_asset_price"] = 1 / ccl_dollar.loc[today_str, "Referencia"]
df2.loc["US$ MEP","this_asset_price"] = 1


In [15]:
for index, row in df2.iterrows():
    if pd.isna(row["this_asset_price"]):
        stock_name = index + "D.BA"
        try:
            price = yf.download(stock_name, period="1d", interval="1d")["Adj Close"].iloc[0]
            df2.loc[index,"this_asset_price"] = price
        except:
            pass


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['AL30D.BA']: Exception('%ticker%: No data found, symbol may be delisted')


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['GD30D.BA']: Exception('%ticker%: No data found, symbol may be delisted')


In [16]:
df2[["Cantidad", "this_asset_price"]]


,Cantidad,this_asset_price
Nombre,,
AR$,1585.74,0.001045
US$ MEP,3744.58,1.000000
SPY,349.00,23.849995
DIA,86.00,18.869000
DISN,17.00,7.535000
INTC,10.00,10.073980
AL30,255.00,NaN
GD30,200.00,NaN


In [ ]:
adsfasdf

In [17]:
t ="""
DIA	Compra	US$ 18,25	38	US$ 693,50	48hs	Ejecutada
AAPL	Compra	US$ 19,30	37	US$ 714,10	48hs	Ejecutada
DIA	Compra	US$ 18,20	52	US$ 946,40	48hs	Ejecutada
DIA	Compra	US$ 18,20	2	US$ 36,40	48hs	Ejecutada
AAPL	Venta	US$ 19,55	84	US$ 1.642,20	48hs	Ejecutada
DIA	Venta	US$ 18,70	54	US$ 0,00	48hs	Cancelada
AMZN	Venta	US$ 1,05	724	US$ 760,20	48hs	Ejecutada
SPY	Venta	US$ 23,70	415	US$ 0,00	48hs	Cancelada
AMZN	Venta	US$ 1,05	724	US$ 0,00	48hs	Cancelada
AMZN	Compra	US$ 1,03	29	US$ 29,87	48hs	Ejecutada
NVDA	Venta	US$ 20,70	1	US$ 20,70	48hs	Ejecutada
SPY	Compra	US$ 23,35	10	US$ 233,50	48hs	Ejecutada
NVDA	Compra	US$ 20,45	9	US$ 0,00	48hs	Cancelada
NVDA	Compra	US$ 20,45	1	US$ 20,45	48hs	Ejecutada
NVDA	Venta	US$ 20,65	12	US$ 247,80	48hs	Ejecutada
NVDA	Compra	US$ 20,45	9	US$ 184,05	48hs	Ejecutada
NVDA	Venta	US$ 20,65	2	US$ 0,00	48hs	Cancelada
NVDA	Venta	US$ 20,65	1	US$ 20,65	48hs	Ejecutada
NVDA	Compra	US$ 20,45	4	US$ 81,80	48hs	Ejecutada
SPY	Compra	US$ 23,35	1	US$ 23,35	48hs	Ejecutada
NVDA	Venta	US$ 20,65	6	US$ 123,90	48hs	Ejecutada
AMZN	Compra	US$ 1,02	156	US$ 0,00	48hs	Cancelada
AMZN	Compra	US$ 1,02	15	US$ 15,30	CI	Ejecutada
NVDA	Venta	US$ 20,90	6	US$ 0,00	48hs	Cancelada
SPY	Venta	US$ 23,55	123	US$ 0,00	CI	Cancelada
SPY	Compra	US$ 23,35	33	US$ 770,55	48hs	Ejecutada
SPY	Compra	US$ 23,35	2	US$ 46,70	48hs	Ejecutada
SPY	Compra	US$ 23,35	2	US$ 46,70	48hs	Ejecutada
SPY	Compra	US$ 23,20	10	US$ 0,00	CI	Cancelada
CVX	Venta	US$ 19,00	2	US$ 38,00	48hs	Ejecutada
SPY	Venta	US$ 23,75	91	US$ 0,00	CI	Cancelada
AAPL	Venta	US$ 19,80	84	US$ 0,00	48hs	Cancelada
SPY	Venta	US$ 23,75	214	US$ 0,00	48hs	Cancelada
SPY	Compra	US$ 23,40	260	US$ 6.084,00	48hs	Ejecutada
AMZN	Venta	US$ 1,04	205	US$ 213,20	CI	Ejecutada
DIA	Venta	US$ 18,80	54	US$ 0,00	48hs	Cancelada
SPY	Venta	US$ 23,80	132	US$ 3.141,60	48hs	Ejecutada
CVX	Venta	US$ 19,90	2	US$ 0,00	48hs	Cancelada
SPY	Compra	US$ 23,30	63	US$ 1.467,90	CI	Ejecutada
"""

In [18]:
data = StringIO(t)
df = pd.read_csv(data, delimiter='\t', header=None)
df

,0,1,2,3,4,5,6
0,DIA,Compra,"US$ 18,25",38,"US$ 693,50",48hs,Ejecutada
1,AAPL,Compra,"US$ 19,30",37,"US$ 714,10",48hs,Ejecutada
2,DIA,Compra,"US$ 18,20",52,"US$ 946,40",48hs,Ejecutada
3,DIA,Compra,"US$ 18,20",2,"US$ 36,40",48hs,Ejecutada
4,AAPL,Venta,"US$ 19,55",84,"US$ 1.642,20",48hs,Ejecutada
5,DIA,Venta,"US$ 18,70",54,"US$ 0,00",48hs,Cancelada
6,AMZN,Venta,"US$ 1,05",724,"US$ 760,20",48hs,Ejecutada
7,SPY,Venta,"US$ 23,70",415,"US$ 0,00",48hs,Cancelada
8,AMZN,Venta,"US$ 1,05",724,"US$ 0,00",48hs,Cancelada
9,AMZN,Compra,"US$ 1,03",29,"US$ 29,87",48hs,Ejecutada


In [19]:
df.columns = ["Especie", "Operación",   "Precio",  "Cantidad",    "Total operación",   "Plazo",   "Estado"]

In [20]:
replacement_mapping = {'Compra': 1, 'Venta': -1}
df['Operación'] = df['Operación'].replace(replacement_mapping)

In [21]:
df["Tot"] = df['Operación'] * df['Cantidad']

In [22]:
df[df["Estado"] == "Ejecutada"].groupby("Especie").agg({"Tot": "sum"})

,Tot
Especie,
AAPL,-47
AMZN,-885
CVX,-2
DIA,92
NVDA,-6
SPY,239


In [23]:
df.dtypes

Especie            object
Operación           int64
Precio             object
Cantidad            int64
Total operación    object
Plazo              object
Estado             object
Tot                 int64
dtype: object